In [38]:
from google.colab import drive;
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!pip install geopandas

In [40]:
#Importaciones--------------------------
import geopandas as gpd
import pandas    as pd
import numpy     as np
import folium

from glob           import glob
from folium         import plugins
from folium.plugins import HeatMap
#---------------------------------------

Este proyecto trata sobre algo que cada vez es más difícil en nuestros días: observar las estrellas.
Debido a la gran contaminación lumínica que hay en todas las ciudades del país, es complicado saber dónde hacer esto.

En primer lugar, me busqué en internet un csv que contuviera la contaminación lumínica que hay en España por ciudades. Como no encontré uno con mucha información sobre el asunto, cambié el enfoque.
Descargué un dataset que tuviera registrada la población por municipios en España, para crear un mapa de calor e indicar las zonas más pobladas del país, indicar mediante un marker, información sobre la contaminación lumínica de la zona.

Mediante scraping, busqué coordenadas donde poder observación de las estrellas en Galicia (y dos lugares de Asturias).
El scraping se hizo en las webs de turismo.gal, caminandoentresenderos.com y mispicaderos.com (esta última por recomendación de un astrónomo).

Saldrán iconos de telescopios, indicando los puntos de observación de estrellas.
Habrá capas de geojson de las comunidades autónomas y los municipios españoles.

Cargar Datasets,Shapefiles e imagenes

Icono telescopio

In [41]:
telescope = '/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/Images/telescope_low.png'

Contaminación lumítica

In [42]:
lp = pd.read_csv("/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/light_pollution.csv")

In [43]:
lp = lp.loc[(lp['Country'] == 'Spain')]

In [44]:
lp.LimitingMag.unique()

array([    6,     4,     3,     2,     1,     5, -9999,     7])

In [45]:
for i in range(8,1,-1):
    lp['LimitingMag'] = lp['LimitingMag'].replace(i,i+1)

In [46]:
lp['LimitingMag'] = lp['LimitingMag'].replace(-9999,1)

In [47]:
#lp['LocationComment'] = lp['LocationComment'].replace(NaN,'Sin Datos')
lp['LocationComment'] = lp['LocationComment'].fillna('Sin Datos')

In [48]:
#lp['SkyComment'] = lp['SkyComment'].replace('NaN','Sin Datos')
lp['SkyComment'] = lp['SkyComment'].fillna('Sin Datos')

In [49]:
display(lp)

,ID,ObsType,ObsID,Latitude,Longitude,Elevation(m),LocalDate,LocalTime,UTDate,UTTime,LimitingMag,SQMReading,SQMSerial,CloudCover,Constellation,SkyComment,LocationComment,Country
0,215886,GAN,14193,28.6378,-17.883300,631.2100,2020-01-01,00:05,2020-01-01,00:05,7,NaN,NaN,clear,Orion,Cielo despejado. No hay contaminacin lumnica.,Zona urbana. 13 grados. Humedad de 60%. Presin...,Spain
1,215887,GAN,14194,28.6607,-17.906900,375366.0000,2020-01-01,00:05,2020-01-01,00:05,5,NaN,NaN,clear,Orion,"Sur, 60 aproximadamente, no se observa la Luna.","Zona urbana, luces de las casas encendidas, fa...",Spain
2,215888,GAN,14195,28.6167,-17.902800,333873.0000,2020-01-01,00:12,2020-01-01,00:12,4,NaN,NaN,clear,Orion,Cielo despejado con gran visibilidad de las es...,"Zona suburbana, con diversos postes de luz y c...",Spain
3,215889,GAN,14198,28.6546,-17.918600,315844.0000,2020-01-01,01:22,2020-01-01,00:22,5,NaN,NaN,1/4 of sky,Orion,Nubes al oeste. El resto del cielo est despeja...,Luces de las farolas de la calle,Spain
4,215893,GAN,14223,28.6546,-17.918600,315202.0000,2020-01-01,22:51,2020-01-01,21:51,3,NaN,NaN,clear,Orion,Temperatura: 14C Humedad: 61 % Presin: 1.027...,Luces de las farolas de la calle,Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28166,244276,GAN,56331,28.6529,-17.905600,365.3500,2020-11-19,22:25,2020-11-19,22:25,5,NaN,NaN,clear,Pegasus,El cielo est totalmente despejado por lo que s...,Nos encontramos ante un espacio urbano donde h...,Spain
28215,244325,GAN,56386,28.6529,-17.905600,365.3500,2020-11-19,22:25,2020-11-19,22:25,5,NaN,NaN,clear,Pegasus,El cielo est totalmente despejado por lo que s...,Nos encontramos ante un espacio urbano donde h...,Spain
28801,244911,GAN,57274,36.8076,-2.573980,15.7416,2020-12-06,19:37,2020-12-07,02:37,3,NaN,NaN,1/4 of sky,Perseus,Sin Datos,Sin Datos,Spain
29434,245544,GAN,58197,40.7338,0.734273,0.0000,2020-12-22,18:20,2020-12-22,17:20,7,NaN,NaN,clear,Perseus,Sin Datos,Sin Datos,Spain


Cargar todas las ciudades de España

In [50]:
spain = pd.read_csv("/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/spain.csv")

In [51]:
locations = list(zip(spain["Latitude"], spain["Longitude"]))

Puntos de Observación de Estrellas

In [52]:
oe = pd.read_csv("/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/puntosObservacion.csv",sep=';')

In [53]:
oe.head(5)

,LUGAR,LATITUDE,LONGITUDE
0,Repibelo,43.326177,-8.517687
1,El Aguilucho,43.359135,-8.839874
2,Campo de baile,42.994038,-8.566976
3,Capilla do Pico Sacro,42.807712,-8.446598
4,Mirador do Eume,43.393447,-8.020458


Municipios

In [54]:
municipios = gpd.read_file("/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/Shapefiles/LineasMunicipales/SHP_ETRS89/ll_municipales_inspire_peninbal_etrs89/ll_municipales_inspire_peninbal_etrs89.shp")

In [55]:
municipios

,INSPIREID,NATIONALCO,COUNTRY,NATLEV,NATLEVNAME,LEGSTATUS,ACCURACY,NAME_BOUND,DATE_BOUND,URL_RESOUR,geometry
0,ES.IGN.BDDAE.M34122727005M34122727019T0223742,M34122727005M34122727019T0223742,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Barreiros#Foz,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-7.26140 43.55113, -7.26222 43.549..."
1,ES.IGN.BDDAE.M34033333057M34033333064T0238007,M34033333057M34033333064T0238007,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Ribera de Arriba#Santo Adriano,2019-11-29,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-5.93887 43.28949, -5.94057 43.291..."
2,ES.IGN.BDDAE.M34074040021M34074040163T0211587,M34074040021M34074040163T0211587,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Arevalillo de Cega#Puebla de Pedraza,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-3.89080 41.19057, -3.89205 41.189..."
3,ES.IGN.BDDAE.M34090808244M34090808263T0241153,M34090808244M34090808263T0241153,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 3 descrito en la hoja registral,Santa Coloma de Cervelló#Sant Vicenç dels Horts,2021-03-08,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (1.99986 41.37159, 2.00201 41.37642..."
4,ES.IGN.BDDAE.M34010404046M34010404057T0200672,M34010404046M34010404057T0200672,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Fondón#Láujar de Andarax,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-2.93042 37.01151, -2.92985 37.009..."
...,...,...,...,...,...,...,...,...,...,...,...
30828,ES.IGN.BDDAE.M34121515016M97999999006T0225750,M34121515016M97999999006T0225750,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,notAgreed,Método 1 descrito en la hoja registral,Camariñas#Océano Atlántico,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-9.10201 43.19078, -9.10194 43.190..."
30829,ES.IGN.BDDAE.M34092525234M34092525904T0217833,M34092525234M34092525904T0217833,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,None,Método 1 descrito en la hoja registral,Tremp#Castell de Mur,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (0.87114 42.14193, 0.86933 42.14275..."
30830,ES.IGN.BDDAE.M34072424070M34072424112T0219416,M34072424070M34072424112T0219416,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Fabero#Peranzanes,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-6.62137 42.83090, -6.62149 42.830..."
30831,ES.IGN.BDDAE.M34074949017M34074949062T0212002,M34074949017M34074949062T0212002,ES,https://inspire.ec.europa.eu/codelist/Administ...,Municipio,agreed,Método 1 descrito en la hoja registral,Asturianos#Espadañedo,2009-07-06,https://centrodedescargas.cnig.es/CentroDescar...,"LINESTRING (-6.43316 42.07706, -6.43398 42.078..."


##Autonomías

In [56]:
autonomias = gpd.read_file("/content/drive/MyDrive/BIDA/Mapas/Trabajo/CSV/Shapefiles/LineasMunicipales//SHP_ETRS89/ll_autonomicas_inspire_peninbal_etrs89/ll_autonomicas_inspire_peninbal_etrs89.shp")

In [57]:
autonomias

,INSPIREID,NATIONALCO,COUNTRY,NATLEV,NATLEVNAME,LEGSTATUS,ACCURACY,NAME_BOUND,DATE_BOUND,URL_RESOUR,geometry
0,ES.IGN.BDDAE.M34120000000M97990000000T0226505,M34120000000M97990000000T0226505,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Galicia#Océano Atlántico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-8.93418 42.38942, -8.93428 42.389..."
1,ES.IGN.BDDAE.M34010000000M98990000000T0225356,M34010000000M98990000000T0225356,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Andalucía#Mar Mediterráneo,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-1.72341 37.29052, -1.72338 37.290..."
2,ES.IGN.BDDAE.M34030000000M99990000000T0224526,M34030000000M99990000000T0224526,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Principado de Asturias#Mar Cantábrico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-6.38645 43.55778, -6.38650 43.557..."
3,ES.IGN.BDDAE.M34120000000M97990000000T0225509,M34120000000M97990000000T0225509,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Galicia#Océano Atlántico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-8.23077 43.46218, -8.23084 43.462..."
4,ES.IGN.BDDAE.M34120000000M97990000000T0226284,M34120000000M97990000000T0226284,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Galicia#Océano Atlántico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-9.02491 42.49263, -9.02500 42.492..."
...,...,...,...,...,...,...,...,...,...,...,...
4118,ES.IGN.BDDAE.M34040000000M98990000000T0224635,M34040000000M98990000000T0224635,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Illes Balears#Mar Mediterráneo,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (2.55212 39.70066, 2.55207 39.70069..."
4119,ES.IGN.BDDAE.M34160000000M99990000000T0224266,M34160000000M99990000000T0224266,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,País Vasco/Euskadi#Mar Cantábrico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-2.41907 43.32737, -2.41902 43.327..."
4120,ES.IGN.BDDAE.M34100000000M98990000000T0228000,M34100000000M98990000000T0228000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Comunitat Valenciana#Mar Mediterráneo,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (0.68969 39.89527, 0.68972 39.89525..."
4121,ES.IGN.BDDAE.M34120000000M97990000000T0225426,M34120000000M97990000000T0225426,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,None,None,Galicia#Océano Atlántico,2009-07-06,https://www.idee.es/csw-inspire-idee/srv/spa/c...,"LINESTRING (-8.06433 43.70215, -8.06438 43.702..."


Capas GeoJson

In [59]:
gjson = {
          #"Municipios" : municipios,
          "Autonomias" : autonomias
        }

Tiles de Folium

In [60]:
tiles = {
           'stamenterrain'        : 'Stamen Terrain',
           'stamentoner'          : 'Stamen Toner',
           'stamenwatercolor'     : 'Stamen WaterColor',
           'cartodbpositron'      : 'CartoDB Positron',
           'cartodbdark_matter'   : 'CartoDB Dark Matter',
        }

In [61]:
m = folium.Map(location=[43, -8.20],zoom_start=9)

#Capas Tiles----------------------
for key, value in tiles.items():
    folium.TileLayer(key,name=value).add_to(m)
#---------------------------------

#Capas Ciudades-------------------
for key, value in gjson.items():
    if key == "Municipios":
        pass
        #descartado por aumentar el peso del fichero
        #folium.GeoJson(value, name=key,show=False).add_to(m)
    else:
        folium.GeoJson(value, name=key,show=True).add_to(m)

#Puntos de Observaciones-------------------------------------------------
for index, punto in oe.iterrows():
    folium.Marker([punto.LATITUDE, punto.LONGITUDE],
    tooltip=punto.LUGAR,
    icon=folium.features.CustomIcon(telescope,icon_size=(32, 32))). \
    add_to(m)
#------------------------------------------------------------------------

#Nivel de Contaminación--------------------------------------------------
for index, punto in lp.iterrows():
    folium.Marker([punto.Latitude, punto.Longitude],popup=(f'{punto.LocationComment} ({punto.SkyComment})'),
    icon=folium.Icon(color='black',icon='cloud', prefix='fa'),
    tooltip="Contaminación lumínica: " + str(punto.LimitingMag)). \
    add_to(m)
#-----------------------------------------------------------------------

HeatMap(locations).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))

folium.LayerControl().add_to(m)
m

In [62]:
m.save('/content/drive/MyDrive/BIDA/Mapas/Trabajo/mapa_contaminacion.html')